In [39]:
from monai.bundle import get_all_bundles_list, get_bundle_info, get_bundle_versions, download
from monai.bundle.config_parser import ConfigParser
from jinja2 import Environment, FileSystemLoader
from bs4 import BeautifulSoup
import markdown
import requests
import json
import tempfile
import os
import re

In [7]:
def _get_all_bundles_info(repo: str = "Project-MONAI/model-zoo", tag: str = "hosting_storage_v1"):
    request_url = f"https://api.github.com/repos/{repo}/releases"
    resp = requests.get(request_url)
    resp.raise_for_status()
    releases_list = json.loads(resp.text)
    bundle_name_pattern = re.compile(r"_v\d*.")
    bundles_info: Dict = {}

    for release in releases_list:
        if release["tag_name"] == tag:
            for asset in release["assets"]:
                asset_name = bundle_name_pattern.split(asset["name"])[0]
                if asset_name not in bundles_info:
                    bundles_info[asset_name] = {}
                asset_version = asset["name"].split(f"{asset_name}_v")[-1].replace(".zip", "")
                bundles_info[asset_name][asset_version] = {
                    "id": asset["id"],
                    "name": asset["name"],
                    "size": asset["size"],
                    "download_count": asset["download_count"],
                    "browser_download_url": asset["browser_download_url"],
                    "created_at": asset["created_at"],
                    "updated_at": asset["updated_at"],
                }
                
    return bundles_info

In [48]:
def get_bundle_web_data(bundle_name, download_dir, all_bundles_info):
    current_bundle = all_bundles_info[bundle_name]
    
    web_data = {}
    web_data["bundle_name"] = bundle_name
    
    download_count = 0
    
    for key, item in current_bundle.items():
        download_count += item["download_count"]
    web_data["downloads"] = download_count
    
    latest_version = sorted(current_bundle.keys())[-1]
    
    if "updated_at" in current_bundle[latest_version]:
        web_data["latest_update"] = current_bundle[latest_version]["updated_at"]
        
    # download zip file and get readme, license, metadata path
    download(name=bundle_name, version=latest_version, bundle_dir=download_dir)

    readme_path = os.path.join(download_dir, bundle_name, "docs", "README.md")
    if os.path.exists(readme_path):
        with open(readme_path, 'r', encoding="utf-8") as f:
            text = f.read()
            readme_html = markdown.markdown(text, extensions=['tables', 'fenced_code'])
            readme_soup = BeautifulSoup(readme_html, "html.parser")
            for img in readme_soup.find_all('img'):
                img_urls = img['src']
                if "https:" not in img_urls or "http:" not in img_urls:
                    img_urls="https://raw.githubusercontent.com/Project-MONAI/model-zoo/dev/models/{bundle_name}/docs/{img_name}".format(bundle_name=bundle_name, img_name=img_urls)
                    img['src'] = img_urls
            
        web_data["readme_path"] = readme_path
        web_data["readme_html"] = readme_soup
        
    license_path = os.path.join(download_dir, bundle_name, "docs", "license.txt")
    if os.path.exists(license_path):
        web_data["license_path"] = license_path
        
    metadata_path = os.path.join(download_dir, bundle_name, "configs", "metadata.json")
    if os.path.exists(metadata_path):
        web_data["metadata"] = ConfigParser.load_config_file(metadata_path)
        
    web_data["download_url"] = current_bundle[latest_version]["browser_download_url"]
    web_data["size"] = current_bundle[latest_version]["size"]
    web_data["image_path"] = "https://raw.githubusercontent.com/Project-MONAI/model-zoo/dev/models/" + bundle_name + "/docs/"        
    
    return web_data

In [35]:
all_bundle_list = _get_all_bundles_info()
temp_dir = tempfile.mkdtemp()

all_models = {}
for item in all_bundle_list.items():
    bundle_name = item[0]
    bundle_web_data = get_bundle_web_data(bundle_name, temp_dir, all_bundle_list)
    all_models[bundle_name] = bundle_web_data

brats_mri_segmentation
lung_nodule_ct_detection
mednist_gan
pancreas_ct_dints_segmentation
pathology_tumor_detection
prostate_mri_anatomy
renalStructures_UNEST_segmentation
spleen_ct_segmentation
spleen_deepedit_annotation
swin_unetr_btcv_segmentation
valve_landmarks
ventricular_short_axis_3label
wholeBrainSeg_Large_UNEST_segmentation
